# Recommender Systems in Python 101
This document will demonstrate how to implement **Collaborative Filtering, Content-Based Filtering and Hybrid mothods** for the task of providing personalized recommendations in Python.

In [2]:
import numpy as np
import pandas as pd
import scipy, math, random, sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt



### Loading data : CI&T Deskdrop dataset
The dataset contains **shared_articles.csv** and **users_interactions.csv**.   

##### shared_articles.csv
It contains information about the articles shared in the platform.
- sharing date, original url, content in plain text, language and author    

Event types at a given timestamp are **CONTENT SHARED** and **CONTENT REMOVED** but only consider here the "CONTENT SHARED" type for the sake of simplicity assuming that all articles were available during the period.

In [5]:
articles_df = pd.read_csv('./archive/shared_articles.csv')
articles_df = articles_df[articles_df['eventType']=='CONTENT SHARED']
articles_df.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en


##### users-interactions.csv
It contains logs of user interactions on shared articles and can be joined to **articles_shared.csv** by **contentId** column.
- VIEW, LIKE, COMMENT CREATED, FOLLOW and BOOKMARK


In [6]:
interactions_df = pd.read_csv('./archive/users_interactions.csv')
interactions_df.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


### Data munging
We associate different interactions types with a weight or strength.   
EX) Assuming that a comment in an article indicates a higher interest of the user than a like or a simple view.

In [14]:
# 각 이벤트에 서로 다른 가중치 배정
event_strength = {'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,}

interactions_df['eventStrength'] = interactions_df['eventType'].apply(lambda x: event_strength[x])

----
### pd.DataFrame.apply()

In [9]:
testdf = pd.DataFrame([[4,'view'],[1,'like'],[2,'comment']],columns=['A','B'])
testdf

,A,B
0,4,view
1,1,like
2,2,comment


In [10]:
teststr = {'view':1, 'like':22, 'comment':44}

In [11]:
testdf['C'] = testdf['B'].apply(lambda x: teststr[x])

In [12]:
testdf

,A,B,C
0,4,view,1
1,1,like,22
2,2,comment,44


---

To avoid the problem of ***user cold-start***, we are keeping in the dataset only users with at least 5 interactions.

In [38]:
users_inter_cnt_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()

In [39]:
print('# users: %d' % len(users_inter_cnt_df))

# users: 1895


In [56]:
users_with_5inter_df = users_inter_cnt_df[users_inter_cnt_df >= 5].reset_index()[['personId']]

In [57]:
print('# users with enough interactions: %d' % len(users_with_5inter_df))

# users with enough interactions: 1140


----

In [51]:
testsize = testdf.groupby(['A','B']).size().groupby(['A']).size()

In [53]:
testsize[testsize>=3]

A
c    3
dtype: int64

In [50]:
testsize[testsize>=3].reset_index()

,A,0
0,c,3


In [54]:
testsize[testsize>=3].reset_index()['A']

0    c
Name: A, dtype: object

In [55]:
testsize[testsize>=3].reset_index()[['A']]

,A
0,c


----

In [58]:
print('# of interactions: %d' % len(interactions_df))
interact_from_selected_users_df = interactions_df.merge(users_with_5inter_df,
                                                       how='right',
                                                       left_on='personId',
                                                       right_on='personId')
print('# of interactions from users with enough interactions: %d' %len(interact_from_selected_users_df))

# of interactions: 72312
# of interactions from users with enough interactions: 69868


To model the user interset on a given article, we aggregate all the interactions the user has performed in an item by weighted sum of interaction type strength and apply a log trans to smooth the distribution.

In [59]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

interactions_full_df = interact_from_selected_users_df.groupby([
    'personId','contentId'])['eventStrength'].sum().apply(smooth_user_preference).reset_index()
print('# of uniq user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of uniq user/item interactions: 39106


,personId,contentId,eventStrength
0,-9223121837663643404,-8949113594875411859,1.000000
1,-9223121837663643404,-8377626164558006982,1.000000
2,-9223121837663643404,-8208801367848627943,1.000000
3,-9223121837663643404,-8187220755213888616,1.000000
4,-9223121837663643404,-7423191370472335463,3.169925
5,-9223121837663643404,-7331393944609614247,1.000000
6,-9223121837663643404,-6872546942144599345,1.000000
7,-9223121837663643404,-6728844082024523434,1.000000
8,-9223121837663643404,-6590819806697898649,1.000000
9,-9223121837663643404,-6558712014192834002,1.584963


### Evaluation
One key aspect of evaluation is to ensure that the trained model generalizes for data it was not trained on, using **Cross-validation** techniques. We are using a simple cross-validation approach named **holdout** in which a random data sample are kept aside in the training process and exclusively used for evaluation.   
All evaluation metrics reported here are computed using the **test set**.      

P.s. train set과 test set을 날짜를 기준으로 구분하여 더 탄탄한 evaluation을 수행할 수 있습니다.   

Evaluation을 위해 사용되는 set metrics 중 **Top-N accuracy metrics**를 사용한다. Test set에서 유저가 실제로 상호작용한 아이템들과 비교하여 유저에게 제안된 상위 추천들의 정확도를 계산한다.   
- For each user
 - For each item the user has interacted in test set
  - 유저가 상호작용하지 않은 100개의 다른 아이템을 샘플링한다.   
  이 아이템들이 유저와 관계가 없다는 단순한 가정을 바탕으로 한다.
  - 하나의 상호작용한 아이템과 100개의 상호작용하지 않은 아이템에서 추천되는 아이템들의 랭크 리스트를 recommender model로 생성한다.
  - 랭크 리스트에서 Top-N accuracy metrics를 계산한다.
- Global Top-N accuracy metrics를 총합한다.   

*Recall@N, NDCG@N, MAP@N*

In [60]:
inter_train_df, inter_test_df = train_test_split(interactions_full_df,
                                                stratify=interactions_full_df['personId'],
                                                test_size=0.2,
                                                random_state=27)

print('# interactions on Train: %d' % len(inter_train_df))
print('# interactions on Test: %d' % len(inter_test_df))

# interactions on Train: 31284
# interactions on Test: 7822


In [61]:
# Indexing by personId to speed up the seaches during evaluaion
interactions_full_df = interactions_full_df.set_index('personId')
inter_train_df = inter_train_df.set_index('personId')
inter_test_df = inter_test_df.set_index('personId')

In [62]:
# 유저와 관련된 아이템 뽑아내기
def get_items_interacted(personid, inter_df):
    interacted_items = inter_df.loc[personid]['contentId']
    return set(interacted_items if type(interacted_items)==pd.Series else [interacted_items])

In [131]:
# Top-N accuracy metrics consts
Eval_Random_Sample_Non_Interacted_Items = 100

class ModelEvaluator:
    # 유저와 관련되지 않은 아이템 중 샘플 뽑아내기
    def get_not_interacted_items_sample(self, personid, sample_size, seed=27):
        interacted_items = get_items_interacted(personid, interactions_full_df)
        all_items = set(articles_df['contentId'])
        non_interacted_items = all_items-interacted_items
        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)
        
    # recommended_items와 item_id는 같은 format    
    def verify_hit_top_n(self, item_id, recommended_items, topn):
        try:
            index = next(i for i,c in enumerate(recommended_items) if c==item_id)
        except:
            index = -1
        # hit = True / False
        hit = int(index in range(0, topn))
        return hit, index
    
    def evaluate_model_for_user(self, model, personid):
        # Getting the items in test set
        interacted_values_testset = inter_test_df.loc[psersonid]
        if type(interacted_values_testset['contentId'])==pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])
        interacted_items_cnt_testset = len(person_interacted_items_testset)
    
        # Getting a ranked recommendation list from a model for a user
        person_recs_df = model.recommend_items(personid, items_to_ignore=get_items_interacted(personid,inter_train_df),
                                               topn=10000000000)
        hits_at_5_cnt = 0
        hits_at_10_cnt = 0
        
        # For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            # Getting a random sample items the user has not interacted
            # represnet items that are assumed to be no relevant to the user
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id,
                                                                               sample_size=Eval_Random_Sample_Non_Interacted_Items,
                                                                               seed=item_id%(2**32))
            
            # Combining the current interactec item with the random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))
            
            # Filtering only recommendations that are either -, -
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]
            valid_recs = valid_recs_df['contentId'].values
            
            # Verifying if the current inter item is among the Top-N
            hits_at_5, index_at_5 = self.verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_cnt += hits_at_5
            
            htis_at_10, index_at_10 = self.verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_cnt += hits_at_10
            
            # Recall is the rate of the interacted items that are ranked among the Top-N
            # when mixed with a set of non-rel items
        recall_at_5 = hits_at_5_cnt / float(interacted_items_cnt_testset)
        recall_at_10 = hits_at_10_cnt / float(interacted_items_cnt_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                      'hits@10_count':hits_at_10_count, 
                      'interacted_count': interacted_items_count_testset,
                      'recall@5': recall_at_5,
                      'recall@10': recall_at_10}
        return person_metrics
            
    def evaluate_model(self, model):
        print('Running evaluation for users')
        people_metrics = []
        
        for idx, personid in enumerate(list(inter_test_df.index.unique().values)):
            if idx % 100 == 0 and idx > 0:
                print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, personid)
            person_metrics['_personid'] = personid
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)
        
        detailed_results_df = pd.DataFrame(people_metrics).sort_values('interacted_count',
                                                                       ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_5 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}
        
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()
        